<a href="https://colab.research.google.com/github/gregorywmorris/MLZoom2022/blob/main/Homework/MLZoomcampWeek5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLZoomcampWeek5

##Homework
In this homework, we will use Credit Card Data from [the previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/04-evaluation/homework.md).

Note: sometimes your answer doesn't match one of the options exactly. That's fine. Select the option that's closest to your solution.

In [1]:
%autosave 1

%load_ext autoreload
%autoreload 2

Autosaving every 1 seconds


In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv'
!wget $data -O data.csv

--2022-10-08 14:38:06--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73250 (72K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  71.53K  --.-KB/s    in 0.01s   

2022-10-08 14:38:06 (5.80 MB/s) - ‘data.csv’ saved [73250/73250]



# Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

**version2022.10.4**

#Question 2
* Use Pipenv to install Scikit-Learn version 1.0.2
* What's the first hash for scikit-learn you get in Pipfile.lock?

**Note:** you should create an empty folder for homework and do it there.

**sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b**

#Models


We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

```
features = ['reports', 'share', 'expenditure', 'owner']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression(solver='liblinear').fit(X, y)

```


**Note:** You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

* DictVectorizer
* LogisticRegression

With wget:

PREFIX=https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework

wget $PREFIX/model1.bin

wget $PREFIX/dv.bin

In [3]:
PREFIX='https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework'

!wget $PREFIX/model1.bin

!wget $PREFIX/dv.bin

--2022-10-08 14:38:06--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889 [application/octet-stream]
Saving to: ‘model1.bin’

model1.bin          100%[===================>]     889  --.-KB/s    in 0s      

2022-10-08 14:38:07 (32.8 MB/s) - ‘model1.bin’ saved [889/889]

--2022-10-08 14:38:07--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.1

#Question 3


Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

`{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}`

What's the probability that this client will get a credit card?

* **0.162**
* ~~0.391~~
* ~~0.601~~
* ~~0.993~~

If you're getting errors when unpickling the files, check their checksum:


```
$ md5sum model1.bin dv.bin
3f57f3ebfdf57a9e1368dcd0f28a4a14  model1.bin
6b7cded86a52af7e81859647fa3a5c2e  dv.bin
```

In [4]:
import pickle

In [5]:
input_file='model1.bin'

In [6]:
dictv = 'dv.bin'

In [7]:
with open(input_file, 'rb') as f_in: 
    model = pickle.load(f_in)

In [8]:
with open(dictv, 'rb') as f_in: 
    dv = pickle.load(f_in)

In [9]:
customer ={"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

In [10]:
X = dv.transform([customer])

In [11]:
y_pred = model.predict_proba(X)[0, 1]
round(y_pred, 3)

0.162

#Question 4

Now let's serve this model as a web service

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using requests:

```
url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit card?

* 0.274
* 0.484
* 0.698
* 0.928

In [22]:
from flask import request, jsonify
import requests

In [14]:
output_file = 'w5hw.bin'
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

In [24]:
url = "http://192.168.1.64:8080"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

ConnectionError: ignored

#Docker

Install Docker. We will use it for the next two questions.

For these questions, we prepared a base image: svizor/zoomcamp-model:3.9.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.9.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

```
FROM python:3.9.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```

We already built it and then pushed it to svizor/zoomcamp-model:3.9.12-slim.

**Note:** You don't need to build this docker image, it's just for your reference.

#Question 5

Download the base image svizor/zoomcamp-model:3.9.12-slim. You can easily make it by using docker pull command.

So what's the size of this base image?

* 15 Mb
* 125 Mb
* 275 Mb
* 415 Mb

You can get this information when running docker images - it'll be in the "SIZE" column.

#Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

```
FROM svizor/zoomcamp-model:3.9.12-slim
# add your stuff here
```

Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with Gunicorn

After that, you can build your docker image.

#Question 6

Let's run your docker container!

After running it, score this client once again:

```
url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()
```



What's the probability that this client will get a credit card now?

* 0.289
* 0.502
* 0.769
* 0.972

#Submit the results
Submit your results here: https://forms.gle/jU2we8f9WeLgX3qa6
You can submit your solution multiple times. In this case, only the last submission will be used
If your answer doesn't match options exactly, select the closest one

##Deadline
The deadline for submitting is 10 October 2022 (Monday), 23:00 CEST (Berlin time).

After that, the form will be closed.